In [2]:
# =========================
# 1. IMPORTS
# =========================
import pandas as pd
import numpy as np
from google.colab import files


# =========================
# 2. UPLOAD FILE
# =========================
print("📤 Upload your CSV or Excel file...")
uploaded = files.upload()

# Get the uploaded file name
file_name = list(uploaded.keys())[0]
print(f"✅ File uploaded: {file_name}")


# =========================
# 3. LOAD DATASET
# =========================
if file_name.endswith(".csv"):
    df = pd.read_csv(file_name)
elif file_name.endswith(".xlsx") or file_name.endswith(".xls"):
    df = pd.read_excel(file_name)
else:
    raise ValueError("❌ Unsupported file format. Upload CSV or Excel only.")

print("\n📄 Raw Data Preview:")
display(df.head())


# =========================
# 4. SMART ANALYZER
# =========================
def smart_analyze_finances(df):
    try:
        # STEP 1: NORMALIZATION
        df = df.copy()
        df.columns = [
            str(c).lower().strip()
            .replace(' ', '_')
            .replace('$', '')
            .replace('₹', '')
            .replace('%', '')
            for c in df.columns
        ]
        print("\n🧹 Normalized Columns:")
        print(df.columns.tolist())

        # STEP 2: INTELLIGENT MAPPING
        mapping = {
            'amount': ["amount", "total", "value", "sum", "amt", "debit", "credit", "billing"],
            'type': ["type", "dr_cr", "income_expense", "transaction_type", "in_out"],
            'category': ["category", "head", "purpose", "expense_type", "expense_category"],
            'tax': ["gst", "tax", "vat", "igst", "cgst", "sgst"],
            'date': ["date", "txn_date", "invoice_date", "posting_date", "transaction_date"]
        }

        found_map = {}
        for target, keys in mapping.items():
            for k in keys:
                if k in df.columns:
                    found_map[target] = k
                    break

        if 'amount' not in found_map:
            print("❌ ERROR: Amount column not found.")
            return

        # STEP 3: CLEANING
        amt_col = found_map['amount']
        df[amt_col] = pd.to_numeric(
            df[amt_col].astype(str).str.replace(r'[$,₹,]', '', regex=True),
            errors='coerce'
        )
        df = df.dropna(subset=[amt_col])

        # STEP 4: TYPE DETECTION
        if 'type' in found_map:
            t_col = found_map['type']
            income_kw = ['income', 'revenue', 'credit', 'sale']
            df['std_type'] = df[t_col].astype(str).str.lower().apply(
                lambda x: 'income' if any(kw in x for kw in income_kw) else 'expense'
            )
        else:
            df['std_type'] = df[amt_col].apply(lambda x: 'income' if x > 0 else 'expense')

        # STEP 5: METRICS
        revenue = df[df['std_type'] == 'income'][amt_col].sum()
        expenses = abs(df[df['std_type'] == 'expense'][amt_col].sum())
        profit = revenue - expenses

        profit_margin = (profit / revenue * 100) if revenue > 0 else 0
        expense_ratio = (expenses / revenue * 100) if revenue > 0 else 0

        # Health Score
        score = 60
        if profit > 0: score += 20
        if revenue > 500000: score += 10
        final_score = min(92, max(0, score))

        # OUTPUT
        print("\n📊 CLEANED DATA SAMPLE:")
        display(df[[amt_col, 'std_type']].head())

        print("\n📈 FINAL METRICS")
        print(f"Total Revenue:   ₹{revenue:,.2f}")
        print(f"Total Expenses:  ₹{expenses:,.2f}")
        print(f"Net Profit:      ₹{profit:,.2f}")
        print(f"Profit Margin:   {profit_margin:.2f}%")
        print(f"Expense Ratio:   {expense_ratio:.2f}%")
        print(f"Health Score:    {final_score}/100")

    except Exception as e:
        print(f"❌ Analysis Failed: {str(e)}")


# =========================
# 5. RUN ANALYSIS
# =========================
smart_analyze_finances(df)


📤 Upload your CSV or Excel file...


Saving sample_financial_data.csv to sample_financial_data (1).csv
✅ File uploaded: sample_financial_data (1).csv

📄 Raw Data Preview:


,Date,Description,Category,Amount,Type
0,2024-01-05,Client A Payment,Sales,15000,Income
1,2024-01-08,Office Rent,Rent,-3500,Expense
2,2024-01-10,Client B Payment,Sales,22000,Income
3,2024-01-12,Inventory Purchase,Inventory,-8000,Expense
4,2024-01-15,Freelance Project,Sales,5000,Income



🧹 Normalized Columns:
['date', 'description', 'category', 'amount', 'type']

📊 CLEANED DATA SAMPLE:


,amount,std_type
0,15000,income
1,-3500,expense
2,22000,income
3,-8000,expense
4,5000,income



📈 FINAL METRICS
Total Revenue:   ₹192,700.00
Total Expenses:  ₹70,700.00
Net Profit:      ₹122,000.00
Profit Margin:   63.31%
Expense Ratio:   36.69%
Health Score:    80/100
